In [1]:
import cv2
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image

In [2]:

# Define the CNN for age prediction
class AgePredictorCNN(nn.Module):
    def __init__(self):
        super(AgePredictorCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.dropout2 = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.dropout3 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc_dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout2(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout3(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc_dropout(x)
        x = self.fc2(x)
        return x


In [3]:
# Load the saved model
model_path = r'D:\ml_projects\haar_caascade\age_predictor_cnn_new.pth'
model = AgePredictorCNN()
model.load_state_dict(torch.load(model_path))
model.eval()

C:\Users\ABHIRAM\AppData\Local\Temp\ipykernel_13264\3867343072.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


AgePredictorCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.25, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout2): Dropout(p=0.25, inplace=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout3): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc_dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [13]:

# Load cascades
face_cascade = cv2.CascadeClassifier('D:\ml_projects\haar_caascade\cascades\haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('D:\ml_projects\haar_caascade\cascades\haarcascade_eye.xml')

# Define transformation with actual input dimensions
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to standard dimensions
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

down code is almost perfect


best code sofar!!!

In [18]:
import cv2
import numpy as np
import torch
from PIL import Image
import time

ages = []
frame_count = 0
max_observations = 19  # Set the number of observations to consider
drowsiness_count = 0   # Counter for drowsiness frames
drowsiness_threshold = 5  # Number of frames to consider as drowsy

# Open the video capture (0 for the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6, minSize=(30, 30))

    # List to hold age predictions for this frame
    frame_ages = []

    # Draw rectangles around the faces
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_roi = gray[y + 10:y + h - 10, x + 25:x + w - 25]

        # Convert the NumPy array (face ROI) to a PIL image
        face_pil = Image.fromarray(face_roi)  # Convert to PIL Image
        
        # Apply transformations to the PIL image
        face_tensor = transform(face_pil).unsqueeze(0)  # Add batch dimension
        
        # Now detect eyes within the detected face region
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.2, minNeighbors=7, minSize=(30, 30))
        
        # Check if no eyes are detected (indicating potential drowsiness)
        if len(eyes) == 0:
            drowsiness_count += 1  # Increment drowsiness counter if no eyes are detected
        else:
            drowsiness_count = 0  # Reset the counter if at least one eye is detected
        
        # Determine if the person is drowsy
        if drowsiness_count > drowsiness_threshold:
            # Predict age if drowsy
            with torch.no_grad():
                age_prediction = model(face_tensor)
            age = int(age_prediction.item())
            
            # Append the predicted age to the frame's list
            frame_ages.append(age)
            ages.append(age)  # Append to the global ages list
            frame_count += 1
            
            # Draw bounding box around the face
            cv2.rectangle(frame, (x + 20, y), (x + w - 20, y + h + 10), (255, 0, 0), 2)
            
            # Draw bounding boxes around eyes if detected
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(frame, (x + ex + 20, y + ey), (x + ex + ew + 20, y + ey + eh), (0, 255, 0), 2)

        else:
            # Reset ages when not drowsy
            ages.clear()  # Clear ages if not drowsy
            frame_count = 0  # Reset frame count when not drowsy

        time.sleep(0.01)

    # Check if we have enough observations for average age
    if frame_count >= max_observations:
        # Calculate average age
        average_age = 1.2 * np.median(ages)
        if average_age >= 40:
            average_age *= 1.3
        
        # Display the average age on each detected face
        for (x, y, w, h) in faces:
            cv2.putText(frame, f"Median Age: {average_age:.1f}", (x + 15, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video - Face Detection and Age Prediction', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()